In [ ]:
!pip3 install lxml

In [43]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import lxml

In [15]:
# Path to chromedriver
# !which chromedriver

In [16]:
# Set the executable path and initialize the chrome browser in splinter

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\EKANGA PATRICK\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [18]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [19]:
#set up the HTML parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [20]:
#begin scraping by chaining '.find' onto previously assigned variable, 'slide_elem'. 
slide_elem.find("div", class_='content_title')


<div class="content_title"><a href="/news/8798/mars-is-getting-a-new-robotic-meteorologist/" target="_self">Mars Is Getting a New Robotic Meteorologist</a></div>

In [21]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

'Mars Is Getting a New Robotic Meteorologist'

In [22]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

"Sensors on NASA's Perseverance will help prepare for future human exploration by taking weather measurements and studying dust particles."

## Scrape Mars Data: Featured Image

### Featured Images

In [23]:
# Set and visit the URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [24]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [25]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [26]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [27]:
# Find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA20465_hires.jpg'

In [28]:
# Use the base URL to create an absolute URL
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA20465_hires.jpg'

## Scrape Mars Data: Mars Facts

In [41]:
!pip3 install lxml

In [44]:
# scrape the entire table with Pandas' .read_html() function
df = pd.read_html('http://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

ImportError: lxml not found, please install it